In [1]:
import numpy as np
from gurobipy import *

In [2]:
with open('UFL.dat') as f:
    dem = np.fromstring(f.readline(), sep=',\t')
    c = np.fromstring(f.readline(), sep=',\t')
    c = c.reshape(len(dem), len(c)//len(dem))
    f = np.fromstring(f.readline(), sep=',\t')

In [3]:
n_clients = dem.shape[0]
print('Demand vector:', dem.shape)
n_facilities = f.shape[0]
print('Fixed-cost vector:', f.shape)
print('Variable-cost matrix:', c.shape)

Demand vector: (45,)
Fixed-cost vector: (40,)
Variable-cost matrix: (45, 40)


In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#     MASTER PROBLEM
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Initialize master problem instance
master = Model('master_problem')

# Add binary variables
z = master.addVar(obj=1, name='z')
y = master.addVars(n_facilities, obj=0, vtype=GRB.BINARY, name='y')
master.update()

# Set objective
master.ModelSense=GRB.MINIMIZE

# Gurobi params
master.setParam('LogToConsole', 0)

# Solve initial bfs 'y'
master.optimize()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#     BENDERS' DECOMPOSITION
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Initialize upper & lower bounds and tolerance
UB = GRB.INFINITY
LB = -1*GRB.INFINITY
tol = 0.1

# Print status
print('\nIteration: 0')
print('UB =', UB, '\nLB =', LB)

# Count parameters for Benders' cuts
feas_cut_count = 0
opt_cut_count = 0

# Begin Benders' Decomposition
while UB-LB > tol:
    
    # Extract 'y' values from master problem solution
    y_bar = np.zeros(n_facilities)
    for i in range(n_facilities):
        y_bar[i] = y[i].X

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #     SUBPROBLEM
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Initialize dual subproblem instance
    sub = Model('subproblem')

    # Add dual variables
    u = sub.addVars(n_clients, name='u')
    v = sub.addVars(n_clients, n_facilities, name='v')
    sub.update()

    # Add dual constraints
    sub.addConstrs((u[i] - v[i,j] <= c[i,j] for i in range(n_clients) for j in range(n_facilities)), name='dual_constr')
    
    # Initialize objective function
    obj_sub = LinExpr()
    for j in range(n_facilities):
        obj_sub += f[j]*y_bar[j]
    for i in range(n_clients):
        obj_sub += dem[i]*u[i]
        for j in range(n_facilities):
            obj_sub += -1*dem[i]*y_bar[j]*v[i,j]
    
    # Set objective
    sub.setObjective(obj_sub, GRB.MAXIMIZE)

    # Gurobi params
    sub.setParam('LogToConsole', 0)
    sub.setParam('InfUnbdInfo', 1)
    sub.setParam('DualReductions', 0)

    # Solve dual subproblem
    sub.optimize()
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #     BENDERS' FEASIBILITY CUT
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    # If subproblem is unbounded
    if sub.status == 5:

        # Extract Unbounded Rays 'u' & 'v' values from solution of subproblem
        u_bar = np.zeros(n_clients)
        for i in range(n_clients):
            u_bar[i] = u[i].UnbdRay

        v_bar = np.zeros((n_clients, n_facilities))
        for i in range(n_clients):
            for j in range(n_facilities):
                v_bar[i,j] = v[i,j].UnbdRay
        
        # Set-up cut
        expr = LinExpr()
        for i in range(n_clients):
            expr += dem[i]*u_bar[i]
            for j in range(n_facilities):
                expr += -1*dem[i]*v_bar[i,j]*y[j]
        
        cut_name = 'benders_feas_cut_' + str(feas_cut_count+1)
        
        # Add cut to master problem
        master.addConstr(expr <= 0, name=cut_name)
        master.update()
        
        # Increase counter
        feas_cut_count +=1

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #     BENDERS' OPTIMALITY CUT
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    # If not unbounded, then subproblem is feasible
    else:
        
        # Extract Extreme Points 'u' & 'v' values from solution of subproblem
        u_bar = np.zeros(n_clients)
        for i in range(n_clients):
            u_bar[i] = u[i].X
        
        v_bar = np.zeros((n_clients, n_facilities))
        for i in range(n_clients):
            for j in range(n_facilities):
                v_bar[i,j] = v[i,j].X
        
        # Set-up cut
        expr = LinExpr()
        for j in range(n_facilities):
            expr += f[j]*y[j]
        for i in range(n_clients):
            expr += dem[i]*u_bar[i]
            for j in range(n_facilities):
                expr += -1*dem[i]*v_bar[i,j]*y[j]
        
        cut_name = 'benders_opt_cut_' + str(opt_cut_count+1)
        
        # Add cut to master problem
        master.addConstr(z >= expr, name=cut_name)
        master.update()
        
        # Increase counter
        opt_cut_count +=1
        
        # Improve upper bound
        UB_bar = 0
        for j in range(n_facilities):
            UB_bar += f[j]*y_bar[j]
        for i in range(n_clients):
            UB_bar += dem[i]*u_bar[i]
            for j in range(n_facilities):
                UB_bar += -1*dem[i]*y_bar[j]*v_bar[i,j]
        
        UB = min(UB, UB_bar)
        
    # Resolve master problem
    master.optimize()
    
    # Improve lower bound
    LB = master.objVal
    
    # Print status
    print('\nIteration:', feas_cut_count+opt_cut_count)
    print('UB =', UB, '\nLB =', LB)

# Print completion status
print('\n  ##########\n     Done!\n  ##########')

Academic license - for non-commercial use only

Iteration: 0
UB = 1e+100 
LB = -1e+100

Iteration: 1
UB = 1e+100 
LB = 0.0

Iteration: 2
UB = 306530.0 
LB = 111530.0

Iteration: 3
UB = 306530.0 
LB = 116530.0

Iteration: 4
UB = 306530.0 
LB = 121530.0

Iteration: 5
UB = 306530.0 
LB = 121530.0

Iteration: 6
UB = 306530.0 
LB = 126530.0

Iteration: 7
UB = 306530.0 
LB = 126530.0

Iteration: 8
UB = 306530.0 
LB = 126530.0

Iteration: 9
UB = 306530.0 
LB = 129806.0

Iteration: 10
UB = 306530.0 
LB = 131530.0

Iteration: 11
UB = 306530.0 
LB = 131530.0

Iteration: 12
UB = 306530.0 
LB = 131530.0

Iteration: 13
UB = 306530.0 
LB = 135187.0

Iteration: 14
UB = 302621.0 
LB = 135884.0

Iteration: 15
UB = 302621.0 
LB = 136530.0

Iteration: 16
UB = 286579.0 
LB = 136530.0

Iteration: 17
UB = 286579.0 
LB = 136530.0

Iteration: 18
UB = 258634.0 
LB = 136530.0

Iteration: 19
UB = 258634.0 
LB = 136530.0

Iteration: 20
UB = 258634.0 
LB = 136530.0

Iteration: 21
UB = 258634.0 
LB = 136530.0

Iter

In [5]:
# Write lp file
master.write('benders_decomposition.lp')